In [1]:
from daq import daqDevice
import daqh
import numpy as np
import time
import serial
import sys
import os
sys.path.append('C:\software_git_repos\cofe-python-analysis-tools\utils_meinhold')
sys.path.append('C:\software_git_repos\cofe-python-analysis-tools\utils_zonca')
sys.path.append('C:\software_git_repos\cofe-python-analysis-tools\utils_zonca\demod')
import cofe_util as cu
sys.path.append('C:\\Anaconda3\\envs\\py27\\Scripts')
import h5py
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def import_data(filename=None):
    '''
    function to read in previously stored data and put in format of freshly read data dictionary
    '''
    
    if filename==None:
        Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
        filename = askopenfilename()
    hf=h5py.File(filename)
    key=hf.keys()[0]
    d={}
    d[key]=squeeze(array(hf[key][0:]))
    hf.close()
    return d

In [3]:
import cofe_util as cu

In [4]:
from Tkinter import Tk
from tkFileDialog import askopenfilename

In [37]:
acqmode=daqh.DaamInfinitePost

In [6]:
def get_data(nchan=4,freq=10,nseconds=5,comment='None',alerts=[58,59,60,118,119,120,178,179,180,238,239,240,298,299,300]):
    """
    function to simply aquire nchan a/d channels at rate freq
    for nseconds seconds
    """
    
    #outdata=np.zeros([nchan,nscans],dtype=float)
    dev=daqDevice('DaqBoard3001USB')
    gains=[]
    flags=[]
    chans=[]
    if nchan > 8:
        uchan=nchan-8
        for i in range(8):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
        for i in range(uchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(256+i)   #HERE is the famous fix where DaqX refs upper level dif channels!
    elif nchan<9:      
        for i in range(nchan):
            gains.append(daqh.DgainX1)
            flags.append(daqh.DafBipolar|daqh.DafDifferential)
            chans.append(i)
    acqmode = daqh.DaamNShot
    dev.AdcSetAcq(acqmode, postTrigCount = nseconds*freq)
    dev.AdcSetScan(chans,gains,flags)
    dev.AdcSetFreq(freq)
    #use the driver buffer here user buffer was very limited (the way I tried anyway) 
    transMask = daqh.DatmUpdateBlock|daqh.DatmCycleOn|daqh.DatmDriverBuf

    buf=dev.AdcTransferSetBuffer(transMask, np.uint(nseconds*freq*nchan))
    #bufMask is for transferring the buffer
    bufMask = daqh.DabtmOldest | daqh.DabtmRetAvail

    timestart = (time.time())
    timenotify = timestart + 5

    dev.SetTriggerEvent(daqh.DatsImmediate,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStartEvent)
    dev.SetTriggerEvent(daqh.DatsScanCount,None, 0, np.array(gains[0],dtype=int), np.array(flags[0],dtype=int), daqh.DaqTypeAnalogLocal, 0, 0, daqh.DaqStopEvent)
    dev.AdcTransferStart()
    dev.AdcArm()
    
    while True:
        alertscopy=alerts[:]
        timenotify = checkAlerts(timenotify, timestart, alerts,alertscopy)        
        
        stat = dev.AdcTransferGetStat()
        active = stat['active']
        if not (active & daqh.DaafAcqActive):
            break
    dev.AdcDisarm()
    outdata,ret=dev.AdcTransferBufData(nseconds*freq, nchan,bufMask)
    
    outdata=np.array(outdata,dtype=float)
    outdata=(outdata-2**15)*20./2**16
    outdata=np.transpose(np.reshape(outdata,[nseconds*freq,nchan]))
    print "Finished collecting data\n----------------------"
    dev.Close()
    

    
    return outdata

In [7]:
def checkAlerts(timeupdate,timestart, alerts, alertscopy, updateincrement=5):
    timecheck = (time.time())
    if timecheck > timeupdate:
        print "Time: " + str(int(timecheck - timestart))
        timeupdate += updateincrement
    for alert in alerts:
        if (timecheck - timestart)%60 > alert:
            print "---- " + str(int(timecheck - timestart)) + " SECONDS ----"
            alerts.remove(alert)
    if 2<(timecheck -timestart%60)<3:
        alerts = alertscopy[:]
        
        
    return timeupdate

In [8]:
def get_date_filename(labelstring=''):
    #version for testing adds an optional string to identify tests
    now=time.localtime()[0:6]
    #dirfmt = "c:\\cofe\\ground_data\\testdata\\%4d_%02d_%02d"
    dirfmt = "C:\\COFE\\IO_3001_USB_acquisition\\data\\%4d_%02d_%02d"
    dirname = dirfmt % now[0:3]
    filefmt = "%02d_%02d_%02d.h5"
    filename= labelstring+filefmt % now[3:6]
    ffilename=os.path.join(dirname,filename)
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    return(ffilename)

In [9]:
#use this function- box is set up with terminations on alternate channels to increase isolation
def get_test(nseconds=10,labelstring='',minfreq=1):
    s=[0,2,4,6]  #active channels, intervening ones are terminated to increase isolation
    R=1 #default GMF
    
    d=get_data(nchan=8,freq=10000,nseconds=nseconds)
    z1=cu.nps(d[2,:],10000,minfreq)
    z2=cu.nps(d[6,:],10000,minfreq)
    #R=np.mean(np.sqrt(z2[1][(z2[0] > 10)& (z2[0] < 50)])/np.sqrt(z1[1][(z1[0] > 10)& (z1[0] < 50)]))
    R=np.mean(np.sqrt(z2[1][(z2[0] > minfreq*10)& (z2[0] < minfreq*100)])/np.sqrt(z1[1][(z1[0] > minfreq*10)& (z1[0] < 100*minfreq)]))
    zd=cu.nps(d[6,:]-R*d[2,:],10000,minfreq)
    figure()
    plot(z1[0],np.sqrt(z1[1]),label='z1-Ref')
    plot(z2[0],np.sqrt(z2[1]),label='z2-Sky')
    plot(z1[0],R*np.sqrt(z1[1]),label='z1-Ref*%s' %str(R))
    plot(zd[0],np.sqrt(zd[1]),label='zdiff Sky-ref*%s' %str(R))
    ylim([0,.05])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    title('Test')
    legend()
    outdata=d[s,:]
    hfile=h5py.File(get_date_filename(labelstring))
    hfile.create_dataset(labelstring,data=[outdata])
    return {labelstring:outdata}

In [10]:
def zplot(d_dict):
    g=1.0e4
    dlabel=d_dict.keys()[0]
    d=d_dict[dlabel]
    z1=cu.nps(d[1,:]/g,10000,.01)
    z2=cu.nps(d[3,:]/g,10000,.01)
    #estimate GMF from a section of spectra 10 hz to 100 hz maybe:
    R=np.mean(np.sqrt(z2[1][(z2[0] > 10)& (z2[0] < 50)])/np.sqrt(z1[1][(z1[0] > 10)& (z1[0] < 50)]))
    zd=cu.nps(d[3,:]/g-R*d[1,:]/g,10000,.01)
    #zdr1=cu.nps(d[3,:]/g-d[1,:]/g,10000)
    figure()
    plot(z1[0],1e9*np.sqrt(z1[1]),label=' z1 Ref')
    plot(z2[0],1e9*np.sqrt(z2[1]),label=' z2 Sky')
    plot(z1[0],R*1e9*np.sqrt(z1[1]),label='ZRef*%s' %str(R))
    #plot(zdr1[0],1e9*np.sqrt(zdr1[1]),label=dlabel+' zdiff Sky-ref')
    plot(zd[0],1e9*np.sqrt(zd[1]),label=dlabel+' zdiff Sky-ref*%s' %str(R))
    #ylim([0,.05])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    title(dlabel)
    legend()

In [11]:
def tryzplot(d_dict,nsteps=5,var=.05,lowpass=None,minfreq=1):
    '''
    this one estmates R then varies +- 5 % around it, first lowpasses the ref if lowpass!=None 
    '''
    samprate=1e4
    g=1.0e4
    dlabel=d_dict.keys()[0]
    d=d_dict[dlabel]
    z1=cu.nps(d[1,:]/g,samprate,minfreq)
    z2=cu.nps(d[3,:]/g,samprate,minfreq)
    #estimate GMF from a section of spectra 10 hz to 100 hz maybe:
    Rnom=np.mean(np.sqrt(z2[1][(z2[0] > minfreq*10)& (z2[0] < minfreq*100)])/np.sqrt(z1[1][(z1[0] > minfreq*10)& (z1[0] < 100*minfreq)]))
    #Rnom=1.0
    figure()
    ref=d[1,:]
    if lowpass!=None:
        ref=cu.lowpass(ref,samprate,lowpass)
        dlabel=dlabel+' Lowpass = %s Hz' %str(lowpass)
    for R in np.linspace(Rnom*(1-var),Rnom*(1+var),nsteps):
        zd=cu.nps(d[3,:]/g-R*ref/g,10000,minfreq)
        plot(zd[0],1e9*np.sqrt(zd[1]),label=dlabel+' zdiff Sky-ref*%s' %str(R))
    plot(z1[0],1e9*np.sqrt(z1[1]),label=dlabel+' zSky raw')        
    #ylim([0,.05])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    title(dlabel)
    legend()

In [31]:
def cmpzplot(d_dict,d_dict2,minfreq=1,labels=None,titletxt='Comparison'):
    '''
    this one estmates R for each of two data dicts and plot raw and diff data 
    labels is a list of strings for the legend
    '''
    samprate=1e4
    g=1.0e4
    figure()
    #first dataset
    dlabel=labels[0]
    if labels==None:
        dlabel=d_dict.keys()[0]
    d=d_dict[d_dict.keys()[0]]
    z1=cu.nps(d[1,:]/g,samprate,minfreq)
    z2=cu.nps(d[3,:]/g,samprate,minfreq)
    #estimate GMF from a section of spectra 10 hz to 100 hz maybe:
    R=np.mean(np.sqrt(z2[1][(z2[0] > minfreq*10)& (z2[0] < minfreq*100)])/np.sqrt(z1[1][(z1[0] > minfreq*10)& (z1[0] < 100*minfreq)]))
    ref=d[1,:]
    zd=cu.nps(d[3,:]/g-R*ref/g,10000,minfreq)
    plot(zd[0],1e9*np.sqrt(zd[1]),label=dlabel+' zdiff Sky-ref*%s' %str(R))
    plot(z2[0],1e9*np.sqrt(z2[1]),label=dlabel+' zSky raw')        
    
    #Second dataset
    dlabel2=labels[1]
    if labels==None:
        dlabel2=d_dict2.keys()[0]
    d=d_dict2[d_dict2.keys()[0]]
    z1=cu.nps(d[1,:]/g,samprate,minfreq)
    z2=cu.nps(d[3,:]/g,samprate,minfreq)
    #estimate GMF from a section of spectra 10 hz to 100 hz maybe:
    R=np.mean(np.sqrt(z2[1][(z2[0] > minfreq*10)& (z2[0] < minfreq*100)])/np.sqrt(z1[1][(z1[0] > minfreq*10)& (z1[0] < 100*minfreq)]))
    ref=d[1,:]
    zd=cu.nps(d[3,:]/g-R*ref/g,10000,minfreq)
    plot(zd[0],1e9*np.sqrt(zd[1]),label=dlabel2+' zdiff Sky-ref*%s' %str(R))
    plot(z2[0],1e9*np.sqrt(z2[1]),label=dlabel2+' zSky raw')        
    
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    title(titletxt)
    legend()

In [32]:
def tryzplotlps(d_dict,nsteps=5,var=.05,lowpasslist=[3000,2000,1000,800,500]):
    '''
    this one estmates R then varies the lowpass via the lowpassliste 
    '''
    minfreq=2
    samprate=1e4
    g=1.0e4
    dlabel=d_dict.keys()[0]
    d=d_dict[dlabel]
    z1=cu.nps(d[1,:]/g,samprate,minfreq)
    z2=cu.nps(d[3,:]/g,samprate,minfreq)
    #estimate GMF from a section of spectra 10 hz to 100 hz maybe:
    Rnom=np.mean(np.sqrt(z2[1][(z2[0] > 10*minfreq)& (z2[0] < 100*minfreq)])/np.sqrt(z1[1][(z1[0] > 10*minfreq)& (z1[0] < 100*minfreq)]))
    #Rnom=1.0
    figure()
    R=Rnom
    for lowpass in lowpasslist:
        ref=cu.lowpass(d[1,:],samprate,lowpass)
        zd=cu.nps(d[3,:]/g-R*ref/g,10000,minfreq)
        plot(zd[0],1e9*np.sqrt(zd[1]),label='lowpass %s Hz' %str(lowpass))
    plot(z1[0],1e9*np.sqrt(z1[1]),label=dlabel+' zSky raw')        
    #ylim([0,.05])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    title(dlabel)
    legend()

In [13]:
def tryzplotg(d_dict,nsteps=5,var=.05):
    '''
    this one estmates R then varies +- 5 % around it 
    '''
    g=1.0e4
    dlabel=d_dict.keys()[0]
    d=d_dict[dlabel]
    z1=cu.nps(d[1,:]/g,10000,.1)
    z2=cu.nps(d[3,:]/g,10000,.1)
    #estimate GMF from a section of spectra 10 hz to 100 hz maybe:
    Rnom=np.mean(np.sqrt(z2[1][(z2[0] > .1)& (z2[0] < 10)])/np.sqrt(z1[1][(z1[0] > .1)& (z1[0] < 10)]))
    figure()
    for R in np.linspace(Rnom*(1-var),Rnom*(1+var),nsteps):
        zd=cu.nps(d[3,:]/g-R*d[1,:]/g,10000,.1)
        plot(zd[0],1e9*np.sqrt(zd[1]),label=dlabel+' zdiff Sky-ref*%s' %str(R))
    #ylim([0,.05])
    xlabel('Frequency, [Hz]')
    ylabel('ASD, [$nV \sqrt{sec}$]')
    title(dlabel)
    legend()

In [16]:
f='C:\\COFE\\IO_3001_USB_acquisition\\data\\2016_09_07\\ref,sky op2711_11_14.h5'

In [16]:
dlab=import_data()

In [53]:
cmpzplot(dlab_refl,dlab_refl26,labels=['reflectiveload17','reflectiveload26'],titletxt='Freq diff comparison, 10 GHz, ambient radiometer')

In [50]:
yscale('log')
xscale('log')

In [51]:
dlab_refl26=get_test(labelstring='ref, hrefl26',nseconds=600)

Time: 5
Time: 10
Time: 15
Time: 20
Time: 25
Time: 30
Time: 35
Time: 40
Time: 45
Time: 50
Time: 55
Time: 60
Time: 65
Time: 70
Time: 75
Time: 80
Time: 85
Time: 90
Time: 95
Time: 100
Time: 105
Time: 110
Time: 115
Time: 120
Time: 125
Time: 130
Time: 135
Time: 140
Time: 145
Time: 150
Time: 155
Time: 160
Time: 165
Time: 170
Time: 175
Time: 180
Time: 185
Time: 190
Time: 195
Time: 200
Time: 205
Time: 210
Time: 215
Time: 220
Time: 225
Time: 230
Time: 235
Time: 240
Time: 245
Time: 250
Time: 255
Time: 260
Time: 265
Time: 270
Time: 275
Time: 280
Time: 285
Time: 290
Time: 295
Time: 300
Time: 305
Time: 310
Time: 315
Time: 320
Time: 325
Time: 330
Time: 335
Time: 340
Time: 345
Time: 350
Time: 355
Time: 360
Time: 365
Time: 370
Time: 375
Time: 380
Time: 385
Time: 390
Time: 395
Time: 400
Time: 405
Time: 410
Time: 415
Time: 420
Time: 425
Time: 430
Time: 435
Time: 440
Time: 445
Time: 450
Time: 455
Time: 460
Time: 465
Time: 470
Time: 475
Time: 480
Time: 485
Time: 490
Time: 495
Time: 500
Time: 505
Time: 510


In [15]:
tryzplot(droof,minfreq=1)

In [41]:
r=roof24522['ref,sky 4522'][1,:]
s=roof24522['ref,sky 4522'][3,:]
figure()
d=s-r*.22484
zr=cu.nps(r,10000,.1)
zs=cu.nps(s,10000,.1)
zd=cu.nps(d,10000,.1)
plot(zr[0],zr[1],label='ref')
plot(zs[0],zs[1],label='sky')
plot(zd[0],zd[1],label='diff')
yscale('log')
xscale('log')
legend()

In [136]:
tryzplotlps(nullterm4522)

0.6
0.4
0.2
0.16
0.1


In [178]:

legend()

In [180]:
rdc=nom4522['ref,sky 4522'][0,:]
sdc=nom4522['ref,sky 4522'][2,:]

In [182]:
(np.mean(rdc)-np.mean(sdc))/np.mean(sdc)

-0.042526756302096597

In [160]:
figure()
x=cu.xps(s,s,10000,.1)
plot(x[0],x[1])

In [171]:
figure()
d=s-r*1.1
d2=s-r*1.2
d3=s-r*1.0
lpd=cu.lowpass(d,10000,20)
lpd2=cu.lowpass(d2,10000,20)
lpd3=cu.lowpass(d3,10000,20)
lpr=cu.lowpass(r,10000,20)
lps=cu.lowpass(s,10000,20)
#plot(r,label='r')
#plot(s,label='s')
#figure()
plot(lpr,label='lpr')
plot(lps,label='lps')
plot(lpd,label='lpdiff')
plot(lpd2,label='lpdiff2')
plot(lpd3,label='lpdiff3')

0.004
0.004
0.004
0.004
0.004


In [23]:
r=teeop27['ref,sky op27'][1,:]
s=teeop27['ref,sky op27'][3,:]

In [34]:
r=tee4522['ref,sky 4522'][1,:]
s=tee4522['ref,sky 4522'][3,:]

In [46]:
zr=cu.nps(r,10000,.1)
zs=cu.nps(s,10000,.1)
zd=cu.nps(s-r*1.1,10000,.1)

In [47]:
figure()
plot(zs[0],zs[1],label='sky')
plot(zr[0],zr[1],label='ref')
plot(zd[0],zd[1],label='diff')
legend()
yscale('log')
xscale('log')


In [29]:
figure()
plot(cu.smooth(r,window_len=150))
plot(cu.smooth(s,window_len=150))
plot(cu.smooth(s-r,window_len=150))

In [15]:
r=jnk2['ref,sky 4522'][1,:]
s=jnk2['ref,sky 4522'][3,:]

In [16]:
figure()
plot(r,s,'.')

In [44]:
figure()
plot(r)
plot(s)
plot(r-s*1.1)

In [19]:
tee4522=tee.copy()

In [49]:
tryzplot(skyrefop27,var=.1,nsteps=6)

In [26]:
figure()
plot(jnk_100s['ref,sky 4522'][1,:])

In [50]:
yscale('log')
xscale('log')

In [90]:
figure()
zplot(par_1M)

In [26]:
ref_sky_4522_1ksec=get_test(labelstring='ref,sky 4522',nseconds=1000)

Time: 5
Time: 10
Time: 15
Time: 20
Time: 25
Time: 30
Time: 35
Time: 40
Time: 45
Time: 50
Time: 55
Time: 60
Time: 65
Time: 70
Time: 75
Time: 80
Time: 85
Time: 90
Time: 95
Time: 100
Time: 105
Time: 110
Time: 115
Time: 120
Time: 125
Time: 130
Time: 135
Time: 140
Time: 145
Time: 150
Time: 155
Time: 160
Time: 165
Time: 170
Time: 175
Time: 180
Time: 185
Time: 190
Time: 195
Time: 200
Time: 205
Time: 210
Time: 215
Time: 220
Time: 225
Time: 230
Time: 235
Time: 240
Time: 245
Time: 250
Time: 255
Time: 260
Time: 265
Time: 270
Time: 275
Time: 280
Time: 285
Time: 290
Time: 295
Time: 300
Time: 305
Time: 310
Time: 315
Time: 320
Time: 325
Time: 330
Time: 335
Time: 340
Time: 345
Time: 350
Time: 355
Time: 360
Time: 365
Time: 370
Time: 375
Time: 380
Time: 385
Time: 390
Time: 395
Time: 400
Time: 405
Time: 410
Time: 415
Time: 420
Time: 425
Time: 430
Time: 435
Time: 440
Time: 445
Time: 450
Time: 455
Time: 460
Time: 465
Time: 470
Time: 475
Time: 480
Time: 485
Time: 490
Time: 495
Time: 500
Time: 505
Time: 510


In [92]:
yscale('log')
xscale('log')
title('1M ohm into parallel SRS @ 1e4, lowpass=1KHz, samplerate 10KHz')

In [62]:
shape(jnk.data())

AttributeError: 'dict' object has no attribute 'data'

In [11]:
hfile.close()

In [21]:
figure(10)
plot(srsf_1M[0])

In [58]:
d=copy(rf_on)
d2=copy(rf_on_2)
r=16.5
diff=d[1]-d[0]*r
zdiff=cu.nps(diff,10000,.1)
diff2=d2[1]-d2[0]*r
zdiff2=cu.nps(diff2,10000,.1)
z1=cu.nps(d[1],10000,.1)
z2=cu.nps(d2[1],10000,.1)
figure()
#plot(z1[0],np.sqrt(z1[1]),label='Ref')
plot(z2[0],np.sqrt(z2[1]),label='Sky')
#plot(z1[0],np.sqrt(z1[1])*15,label='Ref x 15')
plot(zdiff[0],np.sqrt(zdiff[1]),label='Zdiff',color='black')
plot(zdiff2[0],np.sqrt(zdiff2[1]),label='Zdiff2',color='red')
#xscale('log')
#yscale('log')
title('Al foil over window, cover on, wireless router powered')
xlabel('Frequency, [Hz]')
ylabel('ASD, [$nV \sqrt{sec}$]')
legend()

In [22]:
d=copy(par_diodes_1x)
r=.915
diff=d[1]-d[0]*r
zdiff=cu.nps(diff,10000,.1)
z1=cu.nps(d[0],10000,.1)
z2=cu.nps(d[1],10000,.1)
figure()
plot(z1[0],np.sqrt(z1[1]),label='Ref')
plot(z2[0],np.sqrt(z2[1]),label='Sky')
plot(z1[0],np.sqrt(z1[1])*r,label='Ref x %s' %str(r))
plot(zdiff[0],np.sqrt(zdiff[1]),label='Zdiff',color='black')
#xscale('log')
#yscale('log')
xlabel('Frequency, [Hz]')
ylabel('ASD, [$nV \sqrt{sec}$]')
title('Two diodes in parallel (same RF input on a tee)')
legend()

In [33]:
flong=open('router_rf_off_1000sec.pkl','wb')

In [37]:
cPickle.dump(long_rf_off,flong)

In [38]:
flong.close()

In [35]:
import cPickle

In [54]:
rs=16.+arange(10)/5. 
z2=cu.nps(d[1],10000,.01)
figure()
plot(z2[0],np.sqrt(z2[1]),label='Sky')
for r in rs:
    diff=d[1]-d[0]*r
    zdiff=cu.nps(diff,10000,.01)
    plot(zdiff[0],np.sqrt(zdiff[1]),label='Zdiff r=%s' %str(r),lw=2)
xscale('log')
yscale('log')
legend()
xlabel('Frequency, [Hz]')
ylabel('ASD, [$nV \sqrt{sec}$]')
title('Internal difference scaled radiometer ASD, 12/21/15, prm')

In [70]:
r=16.35 
z2=cu.nps(d[1],10000,.01)
figure()
plot(z2[0],np.sqrt(z2[1]),label='Sky')
diff=d[1]-d[0]*r
zdiff=cu.nps(diff,10000,.01)
plot(zdiff[0],np.sqrt(zdiff[1]),label='Zdiff r=%s' %str(r),lw=2)
xscale('log')
yscale('log')
legend()
xlabel('Frequency, [Hz]')
ylabel('ASD, [$nV \sqrt{sec}$ ]')
title('Internal difference scaled radiometer ASD, 12/17/15, prm')

In [90]:
#plot calibrated 1000 sec data set
r=16.35 
z2=cu.nps(d1000sec[1],10000,.001)
figure()
plot(z2[0],1000*cal*np.sqrt(z2[1])/1e4,label='Sky')
diff=d1000sec[1]-d1000sec[0]*r
zdiff=cu.nps(diff,10000,.001)
plot(zdiff[0],1000*cal*np.sqrt(zdiff[1])/1e4,label='Zdiff r=%s' %str(r),lw=2)
xscale('log')
yscale('log')
legend()
xlabel('Frequency, [Hz]')
ylabel('ASD, [$mK \sqrt{sec}$ ]')
title('Internal difference scaled radiometer ASD, 1000 seconds, 12/17/15, prm')

In [91]:
yscale('linear')
xscale('linear')

In [79]:
vsky300=15.8e-3
vsky77=8.66e-3
vref300=0.954e-3
vref77=0.954e-3

In [80]:
y=vsky300/vsky77
tsys=(300-77*y)/(y-1)
print(tsys)
cal=(300-77)/(vsky300-vsky77)

193.473389356


In [96]:
print(500/np.sqrt(3e9))

0.00912870929175


In [59]:
d1000sec=copy(d)

In [72]:
shape(d1000sec)

(2L, 10000000L)

In [28]:

figure()
plot(d300k[1,:]/gain[0],label='ref')
plot(d300k[2,:]/gain[1],label='sky')

legend()

In [11]:
figure()
plot(d1m[:100000]/1e3,label='1 Mohm')
plot(dterm[:100000]/1e4,label='50 ohm term')
legend()

In [21]:
zterm=util.nps(dterm/1.e4,10000,.01)
z1m=util.nps(d1m/1.e3,10000,.01)

In [29]:
Rn=(0.13*np.sqrt(1e6)/np.sqrt(2))

In [31]:
Rn

91.923881554251167

In [43]:
figure()
plot(zterm[0],1e9*sqrt(zterm[1]),label='50 Ohm term')
plot(z1m[0],1e9*sqrt(z1m[1]),label='1 Mohm')
hlines(Rn,min(z1m[0]),max(z1m[0]),color='r',lw=2,zorder=3)
ylabel('ASD, [nV $\sqrt{sec}$ ]')
xlabel('Frequency, [Hz]')
yscale('log')
xscale('log')
legend()